## KNN (K-nearest neighbors) e uma visão inicial sobre problemas de classificação 

<small>Por: Hugo Soares, Cefas Rodrigues e Arilton Filho</small>

### Visão Geral 

O KNN (K-nearest neighbors, <i> K-vizinhos mais próximos</i>) é um dos mais famosos e simples algoritmos de Machine Learning (ML). Ele é o pontapé inicial da maioria dos entusiastas dessa área, pois além de possuir uma implementação simples, também é bastante intuitivo. Quando utilizado para problemas de classificação, ele tem como objetivo classificar uma amostra de classe desconhecida com base na comparação com outros exemplos rotulados (já classificados). Ou seja, ele implementa um aprendizado baseado em instâncias, <i>Instanced Based Learning</i>. 
    <img src="img/Instance-based-Algorithms.png"> 
    <small>Fonte: https://machinelearningmastery.com/a-tour-of-machine-learning-algorithms/</small>

Como ele compara uma amostra não-rotulada com exemplos classificados (já rotulados), dizemos que ele implementa uma aprendizagem supervisionada. Ou seja, o aprendizado do algoritmo se baseia na supervisão de especialistas que anteriormente classificoram os exemplos que servirão de comparação. Assim, cada registro do meu conjunto de dados possui n atributos e uma classe que o rotula. Já a minha amostra possui apenas os atributos. O conjunto de exemplos que utilizamos para realizar as comparações é chamado de Conjunto de Treino (<i>Training Set</i>), logo cada exemplo é chamado de Instância de Treino (<i>Training Instance</i>) 

<img src="img/supervisioned_learning.png">
<small>Fonte: http://bigdata-madesimple.com/machine-learning-explained-understanding-supervised-unsupervised-and-reinforcement-learning/ </small>

### Dataset do estudo de caso

Por exemplo, em nosso estudo de caso utilizaremos o Haberman's Survival Data Set. Ele é composto por 306 registros de um estudo realizado sobre a sobrevida de pacientes submetidos à cirurgia para câncer de mama. Cada registro possui 3 atributos (idade do paciente na época da operação, ano da operação e o número de nódulos axilares positivos detectados) e uma classe que pode ser 1 ou 2 (1 significa que o paciente sobreviveu por 5 anos ou mais e 2 significa que o paciente morreu dentro de 5 anos). 

<img src="img/haberman.png"> 



### Comparação entre a amostra e as intâncias do conjunto de treino

Já sabemos que o KNN se baseia na comparação de uma amostra de entrada com exemplos já rotulados. Mas como ocorre esse comparação? É nesse ponto que reside a simplicidade do algoritmo. Ele somente compara amostra e um exemplo através da <b>distância euclideana</b> entre eles... 
<br>
<p style="font-size:13px">Relembrando, distância euclidiana é a distância entre dois pontos que é definida através do teorema de pitágoras. A distância euclidiana entre a amostra <i>X</i> e o exemplo <i>Y</i> é mostrada abaixo. Onde X1, X2, ..., Xn são os atributos de X e Y1, Y2, ..., Yn são os atributos Y.</p> 

\begin{align}
d(x, y) = \sqrt{ (x_1 - y_1)^2 + (x_2 - y_2)^2 + ... + (x_n - y_n)^2 }
\end{align}

Em seguida ele verifica quais são os K-vizinhos mais próximos da amostra (exemplos com as menores distâncias euclidianas). De acordo com as classes desse vizinhos podemos classificar a amostra. Sendo K=1, verificaremos apenas qual a classe do exemplo mais próximo da entrada e  a indicamos como a possível classe da amostra. Sendo k=3 verificaremos qual a classe que mais se repete nesses 3 vizinhos e a idicamos como classe da amostra. Além disso, podemos indicar essa classe em termos de probabiblidade. Por exemplo, para k=10 se 6 dos vizinhos mais próximos apresentarem a classe A, podemos informar que a amostra tem 60% de chances de pertencer a classe A.  

<img src="img/knn_ilustracao.png"> 
<small>Fonte: https://medium.com/@adi.bronshtein/a-quick-introduction-to-k-nearest-neighbors-algorithm-62214cea29c7</small>

E como podemos analisar a acurácia do nosso algoritmo? Primeiramente dividimos nosso conjunto total de registros rotulados em dois subconjuntos: o conjunto de treino (que já foi explicado) e o conjunto de treino. Em geral, selecionamos randomicamente 70% dos registros para treino e 30% para teste. 
<br><br>
Em seguida, nós aplicamos cada um dos registros de teste ao algoritmo de classificação que construímos e verificamos se a classe gerada pelo algoritmo corresponde a verdadeira classe da instância de teste. Assim, se tivermos 200 exemplos no conjunto de teste e o nosso algoritmo acertar a classe de 190 deles, podemos dizer que sua acurácia é de 95%!  

<img src="img/acuracia.png">

## Vamos Implementar! 

### Preparando nossos dados

Primeiro, importamos nosso dataset de um arquivo externo. Em seguida, o normalizamos, ou seja, deixamos todos os ATRIBUTOS entre 0-1. a coluna da classe permanece igual. Fazemos isso para equilibrar a contribuição de cada atributo no cálculo da distância. Imagine o caso em que o atributo A está no intervalo 0-5 e o atributo B em 0-30000. É fácil perceber que se NÃO houver normalização, possivelmente variações no atributo B terão um impacto muito maior do que variações no atributo A no cálculo da distância entre a amostra e um exemplo qualquer. 

Em seguida, dividimos nosso dataset normalizado em dois subconjuntos: o conjunto de treino e o de teste. Nesse caso, nosso conjunto de treino possui 2/3 do dataset e o restante pertence ao conjunto de treino. 

In [249]:
import numpy as np
from sklearn.preprocessing import normalize

def processar_dataset(nome_arquivo: str):
    """
    Importa os dados de um arquivo e divide o dataset 
    em um conjunto de treino e outro de teste
    :param nome_arquivo: nome do arquivo externo
    """
    dataset = np.loadtxt(nome_arquivo, delimiter=',')
    dataset[:, :3] = normalize(dataset[:,:3]) #Normalizacao apenas dos atributos

    n = dataset.shape[0] #Quantidade de registros do nosso dataset
    np.random.shuffle(dataset)  #Embaralha o dataset 
    conjunto_de_treino = dataset[(n//3):, :] #recebe os elementos com indices > n/3
    conjunto_de_teste = dataset[:(n//3), :] #recebe os elementos com indices < n/3

    print("Dimensão do conjunto de treino:", conjunto_de_treino.shape)
    print("DImensão do conjunto de teste:", conjunto_de_teste.shape)

### Vamos encontrar os vizinhos? 

Agora, criamos uma função que recebe uma amostra e procura no conjunto de treino os k-vizinhos mais próximos. 

In [259]:
from scipy.spatial import distance
import operator

def encontrar_vizinhos(amostra: object, k : int=1):
    """
    Recebe uma amostra e encontra os k-vizinhos mais próximos do conjunto
    treino. 
    
    :param amostra: amostra que deve ser comparada com as instancias
        de treino
    :param k: quantidade de vizinhos selecionados
    :return: array com os k-vizinhos e suas distancias  
    """
    vizinhos_com_distancias = [ [[], 0] ] * conjunto_de_treino.shape[0]
    
    for (index, vizinho) in enumerate(conjunto_de_treino): 
        distancia = distance.euclidean(amostra, vizinho[:3])
        vizinhos_com_distancias[index] = (vizinho, distancia)
        
    vizinhos_com_distancias.sort(key=operator.itemgetter(1))
    
    return vizinhos_com_distancias[:k]

encontrar_vizinhos(conjunto_de_teste[4, :3], 5) # testando com o 4 elemento do conjunto de teste. 
    
    

[(array([70., 58.,  0.,  2.]), 2.0),
 (array([73., 62.,  0.,  1.]), 4.123105625617661),
 (array([70., 58.,  4.,  2.]), 4.47213595499958),
 (array([75., 62.,  1.,  1.]), 5.0990195135927845),
 (array([67., 61.,  0.,  1.]), 5.830951894845301)]

### Escolhendo a classe 

Com base nos vizinhos mais próximos, podemos escolher a classe com a qual rotularemos amostra. O jeito mais simples de fazer isso, é vendo qual a classe que mais se repete nos k-vizinhos mais próximos. Outra forma de fazer isso, seria similar a uma votação com pesos, onde o peso do "voto" de cada vizinho seria inversamente proporcional a distância entre ele e a amostra. 


In [278]:
from collections import OrderedDict

def escolher_classe(k_vizinhos: object):
    """
    :param kvizinhos: os k vizinhos e suas distancias
    :return: retorna com maior ocorrência nos vizinhos
    """
    
    respostas = {}
    for vizinho in k_vizinhos:
        resposta = vizinho[0][-1]
        if resposta in respostas:
            respostas[resposta] += 1 / (1+vizinho[1])
        else:
            respostas[resposta] = 1 / (1 + vizinho[1])
            
    respostas = OrderedDict(sorted(respostas.items(), key=operator.itemgetter(1), reverse=True))
    return list(respostas.items())[0][0]

z = [ ([70., 58.,  0.,  2.], 2.0), ([70., 58.,  0.,  2.], 2.0), 
     ([73., 62.,  0.,  1.], 4.123105625617661),
     ([70., 58.,  4.,  2.], 4.47213595499958) ]

escolher_classe(z)

2.0

### Juntando tudo... 

In [312]:
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt

def main():
    processar_dataset('data/haberman.data')
    respostas_certas = 0 
    
    for amostra in conjunto_de_teste: 
        k_vizinhos = encontrar_vizinhos(amostra[:3], 15)
        classe = escolher_classe(k_vizinhos)
        if amostra[3] == classe: respostas_certas += 1
    
    
    precisao = (respostas_certas/conjunto_de_teste.shape[0])*100
    
    plt.figure(figsize=(200,200))
    vor = Voronoi(conjunto_de_treino[:, 0:2])
    plt.voronoi_plot_2d(vor, show_vertices=0)
    plt.show()
    
    print("Precisão do algoritmo: ", precisao)

In [313]:
main()

Dimensão do conjunto de treino: (204, 4)
DImensão do conjunto de teste: (102, 4)


AttributeError: module 'matplotlib.pyplot' has no attribute 'voronoi_plot_2d'

<Figure size 14400x14400 with 0 Axes>